# Week 4 Implementation: Circle Packing Baseline

Goal: build a reproducible, no-RL baseline for a verifiable optimization loop (generate -> verify -> select -> refine).

## 1) Problem Setup
We pack `N` equal circles of radius `r` inside a unit square, enforcing strict validity constraints.

In [ ]:
import math
import random
import time
from dataclasses import dataclass

SEED = 7
random.seed(SEED)

N_CIRCLES = 18
RADIUS = 0.08
BOX_MIN, BOX_MAX = 0.0, 1.0

ITERATIONS = 80
POOL_SIZE = 64
TOP_K = 8
MUTATION_STD = 0.04

print({
    'seed': SEED,
    'n_circles': N_CIRCLES,
    'radius': RADIUS,
    'iterations': ITERATIONS,
    'pool_size': POOL_SIZE,
    'top_k': TOP_K,
    'mutation_std': MUTATION_STD,
})

## 2) Verifier and Objective
Validity constraints: boundary + non-overlap.

Score (higher is better):
- `-1e6` for invalid candidates (hard rejection)
- for valid candidates, maximize average pairwise center distance (proxy diversity/spread metric).

In [ ]:
@dataclass
class EvalResult:
    valid: bool
    score: float
    boundary_violations: int
    overlap_violations: int

def pairwise_dist(p, q):
    return math.sqrt((p[0] - q[0]) ** 2 + (p[1] - q[1]) ** 2)

def evaluate_candidate(centers, radius):
    boundary_violations = 0
    overlap_violations = 0

    for x, y in centers:
        if x < BOX_MIN + radius or x > BOX_MAX - radius:
            boundary_violations += 1
        if y < BOX_MIN + radius or y > BOX_MAX - radius:
            boundary_violations += 1

    min_sep = 2.0 * radius
    for i in range(len(centers)):
        for j in range(i + 1, len(centers)):
            if pairwise_dist(centers[i], centers[j]) < min_sep:
                overlap_violations += 1

    valid = (boundary_violations == 0 and overlap_violations == 0)

    if not valid:
        return EvalResult(False, -1e6, boundary_violations, overlap_violations)

    if len(centers) < 2:
        spread = 0.0
    else:
        dsum = 0.0
        count = 0
        for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                dsum += pairwise_dist(centers[i], centers[j])
                count += 1
        spread = dsum / count

    return EvalResult(True, spread, 0, 0)

## 3) Candidate Generation and Refinement
Start from random candidates, then mutate top performers.

In [ ]:
def random_candidate(n):
    return [
        (random.uniform(BOX_MIN, BOX_MAX), random.uniform(BOX_MIN, BOX_MAX))
        for _ in range(n)
    ]

def mutate_candidate(centers, std=MUTATION_STD):
    mutated = []
    for x, y in centers:
        nx = x + random.gauss(0.0, std)
        ny = y + random.gauss(0.0, std)
        # Keep points in box; verifier still enforces strict boundary with radius.
        nx = min(max(nx, BOX_MIN), BOX_MAX)
        ny = min(max(ny, BOX_MIN), BOX_MAX)
        mutated.append((nx, ny))
    return mutated

def build_pool(elites, pool_size, n):
    pool = []
    if not elites:
        for _ in range(pool_size):
            pool.append(random_candidate(n))
        return pool

    for _ in range(pool_size):
        if random.random() < 0.25:
            pool.append(random_candidate(n))
        else:
            base = random.choice(elites)
            pool.append(mutate_candidate(base))
    return pool

## 4) Optimization Loop
Tracks best-so-far score, valid rate, and violations.

In [ ]:
history = []
best_candidate = None
best_result = EvalResult(False, -1e9, 0, 0)
elites = []

start = time.time()
for t in range(ITERATIONS):
    pool = build_pool(elites, POOL_SIZE, N_CIRCLES)
    scored = []

    valid_count = 0
    boundary_v = 0
    overlap_v = 0

    for cand in pool:
        result = evaluate_candidate(cand, RADIUS)
        scored.append((result.score, cand, result))
        if result.valid:
            valid_count += 1
        boundary_v += result.boundary_violations
        overlap_v += result.overlap_violations

    scored.sort(key=lambda x: x[0], reverse=True)
    top_items = scored[:TOP_K]
    elites = [item[1] for item in top_items]

    top_score, top_cand, top_eval = top_items[0]
    if top_score > best_result.score:
        best_result = top_eval
        best_candidate = top_cand

    history.append({
        'iter': t,
        'best_score_this_iter': top_score,
        'best_score_so_far': best_result.score,
        'valid_rate': valid_count / POOL_SIZE,
        'boundary_violations': boundary_v,
        'overlap_violations': overlap_v,
    })

elapsed = time.time() - start
print('done in seconds:', round(elapsed, 3))
print('best valid:', best_result.valid)
print('best score:', round(best_result.score, 6))
print('final valid rate:', round(history[-1]['valid_rate'], 3))

## 5) Basic Results and Diagnostics

In [ ]:
for row in history[:5]:
    print(row)
print('...')
for row in history[-5:]:
    print(row)

try:
    import matplotlib.pyplot as plt

    xs = [h['iter'] for h in history]
    ys_best = [h['best_score_so_far'] for h in history]
    ys_valid = [h['valid_rate'] for h in history]

    fig, ax1 = plt.subplots(figsize=(8, 4))
    ax1.plot(xs, ys_best, label='best score so far')
    ax1.set_xlabel('iteration')
    ax1.set_ylabel('score')
    ax1.legend(loc='upper left')

    ax2 = ax1.twinx()
    ax2.plot(xs, ys_valid, color='tab:orange', alpha=0.7, label='valid rate')
    ax2.set_ylabel('valid rate')
    ax2.legend(loc='upper right')

    plt.title('Week 4 Baseline Diagnostics')
    plt.tight_layout()
    plt.show()
except Exception as exc:
    print('matplotlib unavailable or plotting failed:', exc)

## 6) Week 4 Checklist
Fill these after running:
- Best score achieved
- Whether best candidate is valid
- Iteration where best score first appeared
- Average valid rate
- One failure mode and one fix for next week

Planned extension: replace fixed mutate/refine policy with RL-style adaptation and compare sample efficiency under equal compute budget.